In [1]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html

!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-geometric

     |████████████████████████████████| 3.0 MB 4.4 MB/s 
     |████████████████████████████████| 1.6 MB 4.4 MB/s 
     |████████████████████████████████| 222 kB 4.4 MB/s 
     |████████████████████████████████| 376 kB 54.0 MB/s 
     |████████████████████████████████| 45 kB 3.3 MB/s 


In [2]:
import math
import random
import numpy as np
import torch 
import torch_geometric
import torch.nn as nn
import torch.nn.functional as F
import argparse
import torch.optim as optim
import torch_geometric
from torch_geometric.utils import to_dense_adj

In [3]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class GraphConvolution(torch.nn.Module):
    """
    GCN layer
    """
    def __init__(self, in_features, out_features, bias= True):

        super().__init__()

        # dimension of the input features and output features


        self.in_features = in_features
        self.out_features = out_features

        self.weight = None

        # initialise the weight matrix

        stdv = 1 / math.sqrt(self.out_features)

        self.weight = torch.FloatTensor(in_features, out_features).uniform_(-stdv, stdv)

        self.weight = nn.Parameter(self.weight, requires_grad=True)

        if bias:
            self.bias = torch.FloatTensor(out_features).uniform_(-stdv, stdv)
            self.bias = nn.Parameter(self.bias, requires_grad=True)
        else:
            self.bias = None


    def forward(self, fts, adj):

        A_ = adj.to(device) + torch.eye(adj.shape[0]).to(device)
        D_power_  = torch.diag(torch.pow(A_.sum(dim=-1),-0.5))
        support = torch.mm(A_, D_power_)
        support = torch.mm(D_power_, support)

        output = torch.mm(fts, self.weight)
    
        output = torch.sparse.mm(support, output)

        if self.bias is not None:
            return output + self.bias
        else:
            return output

In [4]:


class GCN(torch.nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super().__init__()
        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, fts, adj):
        fts = F.relu(self.gc1(fts, adj))
        fts = F.dropout(fts, self.dropout, training=self.training)
        fts = self.gc2(fts, adj)
        return F.log_softmax(fts, dim=1)

In [5]:

seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train(model, data, num_epochs, use_edge_index=False):
    if not use_edge_index:

        # Create the adjacency matrix
        adj = to_dense_adj(data.edge_index)[0]

    else:

        # Directly use edge_index, ignore this branch for now
        adj = data.edge_index
        
    model.to(device)
    data.to(device)

    # Set up the optimizer
    
    optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

    # A utility function to compute the accuracy
    def get_acc(outs, y, mask):
        return (outs[mask].argmax(dim=1) == y[mask]).sum().float() / mask.sum()

    best_acc_val = -1
    for epoch in range(num_epochs):

        # Zero grads -> forward pass -> compute loss -> backprop
        
        # train mode
        model.train()

        optimizer.zero_grad()
        outs = model(data.x.to(device), adj.to(device))

        # null_loss 

        loss = F.nll_loss(outs[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        # Compute accuracies, print only if this is the best result so far

        # evaluation mode
        model.eval()

        # data.x = the features of the dataset

        outs = model(data.x, adj)

        # validation accuracy 
        acc_val = get_acc(outs, data.y, data.val_mask)

        # test accuracy 
        acc_test = get_acc(outs, data.y, data.test_mask)

        # print the accuracy if it’s incresed
        if acc_val > best_acc_val:
            best_acc_val = acc_val
            print(f'[Epoch {epoch+1}/{num_epochs}] Loss: {loss} | Val: {acc_val:.3f} | Test: {acc_test:.3f}')

    print(f'[Epoch {epoch+1}/{num_epochs}] Loss: {loss} | Val: {acc_val:.3f} | Test: {acc_test:.3f}')
    


Cora = torch_geometric.datasets.Planetoid(root='/', name='Cora')

Citeseer = torch_geometric.datasets.Planetoid(root='/', name='CiteSeer')

Pubmed = torch_geometric.datasets.Planetoid(root='/', name='PubMed')
        
model_cora = GCN(nfeat = Cora.num_features, nhid = 16, nclass = Cora.num_classes, dropout = 0.5)

train(model_cora, data = Cora[0] , num_epochs = 200)


Processing...
Done!
Processing...
Done!
Processing...
Done!
[Epoch 1/200] Loss: 1.9620481729507446 | Val: 0.210 | Test: 0.203
[Epoch 2/200] Loss: 1.8468514680862427 | Val: 0.294 | Test: 0.266
[Epoch 3/200] Loss: 1.7902510166168213 | Val: 0.360 | Test: 0.359
[Epoch 4/200] Loss: 1.7073495388031006 | Val: 0.436 | Test: 0.454
[Epoch 5/200] Loss: 1.5882065296173096 | Val: 0.476 | Test: 0.488
[Epoch 6/200] Loss: 1.5036145448684692 | Val: 0.524 | Test: 0.529
[Epoch 7/200] Loss: 1.4088826179504395 | Val: 0.546 | Test: 0.558
[Epoch 9/200] Loss: 1.1889865398406982 | Val: 0.564 | Test: 0.593
[Epoch 10/200] Loss: 1.1160939931869507 | Val: 0.576 | Test: 0.601
[Epoch 11/200] Loss: 1.068234920501709 | Val: 0.628 | Test: 0.636
[Epoch 12/200] Loss: 0.9679907560348511 | Val: 0.660 | Test: 0.669
[Epoch 13/200] Loss: 0.8586306571960449 | Val: 0.684 | Test: 0.691
[Epoch 14/200] Loss: 0.7883113026618958 | Val: 0.708 | Test: 0.725
[Epoch 15/200] Loss: 0.7423396706581116 | Val: 0.736 | Test: 0.745
[Epoch 16/2

In [6]:
        
model_cit = GCN(nfeat = Citeseer.num_features, nhid = 16, nclass = Citeseer.num_classes, dropout = 0.5)

train(model_cit, data = Citeseer[0] , num_epochs = 200)

[Epoch 1/200] Loss: 1.9599077701568604 | Val: 0.246 | Test: 0.246
[Epoch 2/200] Loss: 1.7302590608596802 | Val: 0.340 | Test: 0.342
[Epoch 3/200] Loss: 1.5734277963638306 | Val: 0.398 | Test: 0.399
[Epoch 4/200] Loss: 1.439060926437378 | Val: 0.452 | Test: 0.457
[Epoch 5/200] Loss: 1.2783361673355103 | Val: 0.494 | Test: 0.514
[Epoch 6/200] Loss: 1.1819312572479248 | Val: 0.534 | Test: 0.545
[Epoch 7/200] Loss: 1.0653191804885864 | Val: 0.570 | Test: 0.576
[Epoch 8/200] Loss: 0.9066251516342163 | Val: 0.592 | Test: 0.600
[Epoch 9/200] Loss: 0.8510716557502747 | Val: 0.616 | Test: 0.608
[Epoch 10/200] Loss: 0.7160793542861938 | Val: 0.622 | Test: 0.619
[Epoch 11/200] Loss: 0.6603959202766418 | Val: 0.626 | Test: 0.631
[Epoch 13/200] Loss: 0.5386476516723633 | Val: 0.632 | Test: 0.643
[Epoch 14/200] Loss: 0.45930805802345276 | Val: 0.636 | Test: 0.643
[Epoch 15/200] Loss: 0.3511947691440582 | Val: 0.638 | Test: 0.647
[Epoch 16/200] Loss: 0.37237781286239624 | Val: 0.642 | Test: 0.641
[Ep

In [8]:
model_pub = GCN(nfeat = Pubmed.num_features, nhid = 16, nclass = Pubmed.num_classes, dropout = 0.5)

train(model_pub, data = Pubmed[0] , num_epochs = 200)